# Assignment 1

Date: 03-10-2020 <br>
Nick Radunovic (s2072724) <br>
Cheyenne Heath (s1647865) <br>

Tasks:
1. Download W-NUT_data.zip from the Brightspace assignment and unzip the directory. It
contains 3 IOB files: wnut17train.conll (train), emerging.dev.conll (dev),
emerging.test.annotated (test)
2. The IOB files do not contain POS tags yet. Add a function to your CRFsuite script that reads
the IOB files and adds POS tags (using an existing package for linguistic processing such as
Spacy or NLTK). The data needs to be stored in the same way as the benchmark data from
the tutorial (an array of triples (word,pos,biotag)).
3. Run a baseline run (train -> test) with the features directly copied from the tutorial.
4. Set up hyperparameter optimization using the dev set and evaluate the result on the test set.
5. Extend the features: add a larger context (-2 .. +2 or more) and engineer a few other features
that might be relevant for this task. Have a look at the train/dev data to get inspiration on
potentially relevant papers.
6. Experiment with the effect of different feature sets on the quality of the labelling.

In [1]:
#Imports
import numpy as np
from itertools import chain
from collections import Counter
import eli5

import nltk
nltk.download('averaged_perceptron_tagger')
import sklearn
import scipy.stats
from sklearn.metrics import make_scorer
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RandomizedSearchCV

import sklearn_crfsuite
from sklearn_crfsuite import scorers
from sklearn_crfsuite import metrics

C:\Users\Stand\anaconda3\lib\site-packages\sklearn\utils\deprecation.py:144: FutureWarning: The sklearn.metrics.scorer module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.metrics. Anything that cannot be imported from sklearn.metrics is now part of the private API.
  warnings.warn(message, FutureWarning)
C:\Users\Stand\anaconda3\lib\site-packages\sklearn\utils\deprecation.py:144: FutureWarning: The sklearn.feature_selection.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.feature_selection. Anything that cannot be imported from sklearn.feature_selection is now part of the private API.
  warnings.warn(message, FutureWarning)
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\Stand\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perce

Function to read in the data per sentence. 

In [2]:
def parse_data(file):    
    sents = []
    with open(file, encoding='utf-8') as fp:
        new_sent = []
        for line in fp:
            if (line == '\n') or (line == '\t\n'):
                #new line so end of sentence, append new_sent to sents array and clear the new_sent
                sents.append(new_sent)
                new_sent = []
            else:
                #create tuple and add to sentence
                new_line = line.strip()
                new_sent.append(tuple(new_line.split('\t')))
    return sents

In [3]:
#parse all files
train_sents = parse_data('wnut17train.conll')
dev_sents = parse_data('emerging.dev.conll')
test_sents = parse_data('emerging.test.annotated')

dev_sents[0]

[('Stabilized', 'O'),
 ('approach', 'O'),
 ('or', 'O'),
 ('not', 'O'),
 ('?', 'O'),
 ('That', 'O'),
 ('´', 'O'),
 ('s', 'O'),
 ('insane', 'O'),
 ('and', 'O'),
 ('good', 'O'),
 ('.', 'O')]

In [4]:
def add_POS_tag(word_tuple):
    #convert tuple to list
    l = list(word_tuple)
    
    #insert new value at index 1
    new_val = nltk.pos_tag(word_tuple)
    l.insert(1, new_val[0][1])
    
    #convert list again to tuple
    new_word_tuple = tuple(l)
    return new_word_tuple  

In [5]:
#add pos tag to each dataset, can take a few minutes
train_sents = [[add_POS_tag(word) for word in sentence] for sentence in train_sents]
dev_sents = [[add_POS_tag(word) for word in sentence] for sentence in dev_sents]
test_sents = [[add_POS_tag(word) for word in sentence] for sentence in test_sents]

dev_sents[3]

[('wow', 'NN', 'O'),
 ('emma', 'JJ', 'B-person'),
 ('and', 'CC', 'O'),
 ('kaite', 'VB', 'B-person'),
 ('is', 'VBZ', 'O'),
 ('so', 'RB', 'O'),
 ('very', 'RB', 'O'),
 ('cute', 'NN', 'O'),
 ('and', 'CC', 'O'),
 ('so', 'RB', 'O'),
 ('funny', 'JJ', 'O'),
 ('😀', 'NN', 'O'),
 ('😀', 'NN', 'O'),
 ('😀', 'NN', 'O'),
 ('😗', 'NN', 'O'),
 ('😘', 'NN', 'O'),
 ('i', 'NN', 'O'),
 ('wish', 'JJ', 'O'),
 ('im', 'NN', 'O'),
 ('ryan', 'JJ', 'B-person'),
 ('😭', 'NN', 'O'),
 ('😭', 'NN', 'O'),
 ('😭', 'NN', 'O')]

3. Run a baseline run (train -> test) with the features directly copied from the tutorial.

In [6]:
def word2features(sent, i):
    word = sent[i][0]
    postag = sent[i][1]

    features = {
        'bias': 1.0,
        'word.lower()': word.lower(),
        'word[-3:]': word[-3:],
        'word[-2:]': word[-2:],
        'word.isupper()': word.isupper(),
        'word.istitle()': word.istitle(),
        'word.isdigit()': word.isdigit(),
        'postag': postag,
        'postag[:2]': postag[:2],
    }
    if i > 0:
        word1 = sent[i-1][0]
        postag1 = sent[i-1][1]
        features.update({
            '-1:word.lower()': word1.lower(),
            '-1:word.istitle()': word1.istitle(),
            '-1:word.isupper()': word1.isupper(),
            '-1:postag': postag1,
            '-1:postag[:2]': postag1[:2],
        })
    else:
        features['BOS'] = True

    if i < len(sent)-1:
        word1 = sent[i+1][0]
        postag1 = sent[i+1][1]
        features.update({
            '+1:word.lower()': word1.lower(),
            '+1:word.istitle()': word1.istitle(),
            '+1:word.isupper()': word1.isupper(),
            '+1:postag': postag1,
            '+1:postag[:2]': postag1[:2],
        })
    else:
        features['EOS'] = True

    return features

def sent2features(sent):
    return [word2features(sent, i) for i in range(len(sent))]

def sent2labels(sent):
    return [label for token, postag, label in sent]

def sent2tokens(sent):
    return [token for token, postag, label in sent]

In [7]:
X_train = [sent2features(s) for s in train_sents]
y_train = [sent2labels(s) for s in train_sents]

X_dev = [sent2features(s) for s in dev_sents]
y_dev = [sent2labels(s) for s in dev_sents]

X_test = [sent2features(s) for s in test_sents]
y_test = [sent2labels(s) for s in test_sents]

In [8]:
crf = sklearn_crfsuite.CRF(
    algorithm='lbfgs',
    c1=0.1,
    c2=0.1,
    max_iterations=100,
    all_possible_transitions=True
)
crf.fit(X_dev, y_dev)

C:\Users\Stand\anaconda3\lib\site-packages\sklearn\base.py:197: FutureWarning: From version 0.24, get_params will raise an AttributeError if a parameter cannot be retrieved as an instance attribute. Previously it would return None.
  FutureWarning)


CRF(algorithm='lbfgs', all_possible_states=None, all_possible_transitions=True,
    averaging=None, c=None, c1=0.1, c2=0.1, calibration_candidates=None,
    calibration_eta=None, calibration_max_trials=None, calibration_rate=None,
    calibration_samples=None, delta=None, epsilon=None, error_sensitive=None,
    gamma=None, keep_tempfiles=None, linesearch=None, max_iterations=100,
    max_linesearch=None, min_freq=None, model_filename=None, num_memories=None,
    pa_type=None, period=None, trainer_cls=None, variance=None, verbose=False)

In [9]:
labels = list(crf.classes_)
labels.remove('O')
labels

['B-location',
 'I-location',
 'B-creative-work',
 'I-creative-work',
 'B-person',
 'I-person',
 'B-product',
 'I-product',
 'B-group',
 'I-group',
 'B-corporation',
 'I-corporation']

In [10]:
y_pred = crf.predict(X_test)
print(metrics.flat_f1_score(y_test, y_pred,
                      average='weighted', labels=labels))

sorted_labels = sorted(
    labels,
    key=lambda name: (name[1:], name[0])
)
print(metrics.flat_classification_report(
    y_test, y_pred, labels=sorted_labels, digits=3
))

0.21761899424561096
                 precision    recall  f1-score   support

  B-corporation      0.000     0.000     0.000        66
  I-corporation      0.000     0.000     0.000        22
B-creative-work      0.164     0.077     0.105       142
I-creative-work      0.217     0.211     0.214       218
        B-group      0.000     0.000     0.000       165
        I-group      0.000     0.000     0.000        70
     B-location      0.422     0.180     0.252       150
     I-location      0.316     0.064     0.106        94
       B-person      0.480     0.410     0.442       429
       I-person      0.467     0.374     0.415       131
      B-product      0.162     0.087     0.113       127
      I-product      0.107     0.071     0.086       126

      micro avg      0.337     0.193     0.245      1740
      macro avg      0.194     0.123     0.144      1740
   weighted avg      0.267     0.193     0.218      1740



Set up hyperparameter optimization using the dev set and evaluate the result on the test set.

In [11]:
# define fixed parameters and parameters to search
crf = sklearn_crfsuite.CRF(
    algorithm='lbfgs', 
    max_iterations=100, 
    all_possible_transitions=True
)
params_space = {
    'c1': scipy.stats.expon(scale=0.5),
    'c2': scipy.stats.expon(scale=0.05),
}

# use the same metric for evaluation
f1_scorer = make_scorer(metrics.flat_f1_score, 
                        average='weighted', labels=labels)

# search
rs = RandomizedSearchCV(crf, params_space, 
                        cv=3, 
                        verbose=1, 
                        n_jobs=-1, 
                        n_iter=10, 
                        scoring=f1_scorer)
rs.fit(X_train, y_train)

Fitting 3 folds for each of 10 candidates, totalling 30 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done  30 out of  30 | elapsed:  1.9min finished


RandomizedSearchCV(cv=3, error_score=nan,
                   estimator=CRF(algorithm='lbfgs', all_possible_states=None,
                                 all_possible_transitions=True, averaging=None,
                                 c=None, c1=None, c2=None,
                                 calibration_candidates=None,
                                 calibration_eta=None,
                                 calibration_max_trials=None,
                                 calibration_rate=None,
                                 calibration_samples=None, delta=None,
                                 epsilon=None, error_sensitive=None, gamma=None,
                                 keep_...
                                        'c2': <scipy.stats._distn_infrastructure.rv_frozen object at 0x00000117B6545B48>},
                   pre_dispatch='2*n_jobs', random_state=None, refit=True,
                   return_train_score=False,
                   scoring=make_scorer(flat_f1_score, average=weighte

In [12]:
print('best params:', rs.best_params_)
print('best CV score:', rs.best_score_)
print('model size: {:0.2f}M'.format(rs.best_estimator_.size_ / 1000000))

best params: {'c1': 0.07300010191128617, 'c2': 0.008177381638813945}
best CV score: 0.40490953073203045
model size: 0.68M


#### Extend the features

This is the extended word2feature function that encompasses both a bigger range (-3 to +3) and a new feature: 'word.starts_with_uppercase'.

In [18]:
def word2features_extended(sent, i):
    word = sent[i][0]
    postag = sent[i][1]

    features = {
        'bias': 1.0,
        'word.lower()': word.lower(),
        'word[-3:]': word[-3:],
        'word[-2:]': word[-2:],
        'word.isupper()': word.isupper(),
        'word.istitle()': word.istitle(),
        'word.isdigit()': word.isdigit(),
        #'word.starts_with_uppercase': word[:1].isupper(),
        'postag': postag,
        'postag[:2]': postag[:2],
    }
    if i > 2:
        word3 = sent[i-3][0]
        postag3 = sent[i-3][1]
        features.update({
            '-3:word.lower()': word3.lower(),
            '-3:word.istitle()': word3.istitle(),
            '-3:word.isupper()': word3.isupper(),
            '-3:word.isdigit()': word3.isdigit(),
            '-3:postag': postag3,
            '-3:postag[:2]': postag3[:2],
        })
    if i > 1:
        word2 = sent[i-2][0]
        postag2 = sent[i-2][1]
        features.update({
            '-2:word.lower()': word2.lower(),
            '-2:word.istitle()': word2.istitle(),
            '-2:word.isupper()': word2.isupper(),
            '-2:word.isdigit()': word2.isdigit(),
            '-2:postag': postag2,
            '-2:postag[:2]': postag2[:2],
        })
    if i > 0:
        word1 = sent[i-1][0]
        postag1 = sent[i-1][1]
        features.update({
            '-1:word.lower()': word1.lower(),
            '-1:word.istitle()': word1.istitle(),
            '-1:word.isupper()': word1.isupper(),
            #'word.starts_with_uppercase': word1[:1].isupper(),
            '-1:word.isdigit()': word1.isdigit(),
            '-1:postag': postag1,
            '-1:postag[:2]': postag1[:2],
        })
    else:
        features['BOS'] = True

    if i < len(sent)-3:
        word3 = sent[i+3][0]
        postag3 = sent[i+3][1]
        features.update({
            '+3:word.lower()': word3.lower(),
            '+3:word.istitle()': word3.istitle(),
            '+3:word.isupper()': word3.isupper(),
            '+3:word.isdigit()': word3.isdigit(),
            '+3:postag': postag3,
            '+3:postag[:2]': postag3[:2],
        })
    if i < len(sent)-2:
        word2 = sent[i+2][0]
        postag2 = sent[i+2][1]
        features.update({
            '+2:word.lower()': word2.lower(),
            '+2:word.istitle()': word2.istitle(),
            '+2:word.isupper()': word2.isupper(),
            '+2:word.isdigit()': word2.isdigit(),
            '+2:postag': postag2,
            '+2:postag[:2]': postag2[:2],
        })
    if i < len(sent)-1:
        word1 = sent[i+1][0]
        postag1 = sent[i+1][1]
        features.update({
            '+1:word.lower()': word1.lower(),
            '+1:word.istitle()': word1.istitle(),
            '+1:word.isupper()': word1.isupper(),
            '+1:word.isdigit()': word1.isdigit(),
            '+1:postag': postag1,
            '+1:postag[:2]': postag1[:2],
        })
    else:
        features['EOS'] = True

    return features

def sent2features_extended(sent):
    return [word2features_extended(sent, i) for i in range(len(sent))]

In [19]:
X_dev = [sent2features_extended(s) for s in dev_sents]
y_dev = [sent2labels(s) for s in dev_sents]

X_test = [sent2features_extended(s) for s in test_sents]
y_test = [sent2labels(s) for s in test_sents]

In [20]:
crf = sklearn_crfsuite.CRF(
    algorithm='lbfgs',
    c1=0.1,
    c2=0.1,
    max_iterations=100,
    all_possible_transitions=True
)
crf.fit(X_dev, y_dev)

y_pred = crf.predict(X_test)

sorted_labels = sorted(
    labels,
    key=lambda name: (name[1:], name[0])
)

print("weighted avg:")
weighted_avg = metrics.flat_classification_report(
    y_test, y_pred, labels=sorted_labels, digits=3, output_dict=True)['weighted avg']
for k in weighted_avg.keys():
    print("%s: %s" % (k, weighted_avg[k]))

weighted avg:
precision: 0.28967513458532324
recall: 0.17413793103448275
f1-score: 0.20047156042488948
support: 1740


C:\Users\Stand\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


However, note that the F1 score hasn't gotten any higher after using the extended features. 

## TO-DO
Adjusting the features such that the F1 score is as high as possible. 
I don't get why the F1 score is so low, compared to that obtained in the tutorial :(

In [16]:
# define fixed parameters and parameters to search
crf = sklearn_crfsuite.CRF(
    algorithm='lbfgs', 
    max_iterations=100, 
    all_possible_transitions=True
)
params_space = {
    'c1': scipy.stats.expon(scale=0.5),
    'c2': scipy.stats.expon(scale=0.05),
}

# use the same metric for evaluation
f1_scorer = make_scorer(metrics.flat_f1_score, 
                        average='weighted', labels=labels)

# search
rs = RandomizedSearchCV(crf, params_space, 
                        cv=3, 
                        verbose=1, 
                        n_jobs=-1, 
                        n_iter=10, 
                        scoring=f1_scorer)
rs.fit(X_dev, y_dev)

print('best params:', rs.best_params_)
print("\nweighted avg:")
crf = rs.best_estimator_
y_pred = crf.predict(X_test)
weighted_avg = metrics.flat_classification_report(
    y_test, y_pred, labels=sorted_labels, digits=3, output_dict=True)['weighted avg']
for k in weighted_avg.keys():
    print("%s: %s" % (k, weighted_avg[k]))

Fitting 3 folds for each of 10 candidates, totalling 30 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done  30 out of  30 | elapsed:   55.0s finished


best params: {'c1': 0.007359360551051685, 'c2': 0.06400853762971355}

weighted avg:
precision: 0.2750374923283246
recall: 0.15804597701149425
f1-score: 0.19011907932652336
support: 1740


In [17]:
# For interpretation see: https://eli5.readthedocs.io/en/latest/tutorials/sklearn_crfsuite.html
eli5.show_weights(crf, top=30)

C:\Users\Stand\anaconda3\lib\site-packages\sklearn\base.py:197: FutureWarning: From version 0.24, get_params will raise an AttributeError if a parameter cannot be retrieved as an instance attribute. Previously it would return None.
  FutureWarning)


From \ To,O,B-corporation,I-corporation,B-creative-work,I-creative-work,B-group,I-group,B-location,I-location,B-person,I-person,B-product,I-product
O,2.817,0.126,-1.673,0.805,-2.881,1.05,-2.413,1.292,-2.297,1.803,-2.941,1.332,-2.58
B-corporation,-0.004,0.748,2.611,-0.366,-0.258,-0.201,-0.252,-0.169,-0.23,-0.475,-0.355,0.138,-0.389
I-corporation,-0.151,-0.036,1.737,-0.144,-0.102,-0.05,-0.098,-0.054,-0.062,-0.263,-0.082,-0.065,-0.07
B-creative-work,-0.374,-0.16,-0.268,-0.585,5.13,-0.239,-0.317,-0.135,-0.366,-0.771,-0.553,-0.348,-0.427
I-creative-work,0.108,-0.163,-0.21,-0.018,4.842,-0.275,-0.435,-0.379,-0.437,-0.866,-0.431,-0.247,-0.306
B-group,-0.508,-0.118,-0.088,-0.238,-0.302,-0.325,3.846,-0.119,-0.237,-0.488,-0.331,-0.157,-0.31
I-group,-0.555,-0.088,-0.064,-0.172,-0.206,-0.296,3.089,-0.052,-0.232,-0.442,-0.155,-0.106,-0.078
B-location,-0.193,-0.243,-0.234,0.131,-0.492,-0.224,-0.301,-0.351,3.884,-0.853,-0.581,-0.303,-0.418
I-location,-0.274,-0.088,-0.042,-0.127,-0.169,-0.078,-0.117,-0.463,2.939,-0.337,-0.16,-0.086,-0.128
B-person,0.951,-0.479,-0.746,-0.715,-0.858,-0.367,-0.634,-0.407,-0.61,-0.792,4.818,-0.677,-1.028


Maybe, we could find out what features contribute to the low F1 score by looking into the table above!